In [36]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import csv
from scipy import stats
from scipy.stats import linregress
from datetime import date
import scipy.stats as st
from sqlalchemy import create_engine
pd.options.display.float_format = '{:.2f}'.format

In [37]:
gdp_data = pd.read_csv("Resources/gdp_data.csv")
life_expectancy = pd.read_csv("Resources/life_expectancy.csv")

In [38]:
#Use case for recession analysis - 2008 + 3 years = 2008 - 2011
#Cleaning data to include only country, country code, and applicable years for GDP.
gdp_df = gdp_data.loc[:,["Country Name", "Country Code", "2008","2009","2010","2011"]]
gdp_df["2008"] = gdp_df["2008"].astype(float)
gdp_df["2009"] = gdp_df["2009"].astype(float)
gdp_df["2010"] = gdp_df["2010"].astype(float)
gdp_df["2011"] = gdp_df["2011"].astype(float)
gdp_df = gdp_df.dropna()
gdp_df

,Country Name,Country Code,2008,2009,2010,2011
0,Aruba,ABW,2745251396.65,2498882681.56,2390502793.30,2549720670.39
1,Afghanistan,AFG,10109218067.79,12439087076.77,15856574731.44,17804280538.11
2,Angola,AGO,88538611205.14,70307163678.24,83799496611.20,111789686464.44
3,Albania,ALB,12881353507.85,12044208085.86,11926953213.70,12890866742.65
4,Andorra,AND,4007353156.58,3660530702.97,3355695364.24,3442062830.14
...,...,...,...,...,...,...
259,Kosovo,XKX,5687417606.56,5653792720.20,5830463576.16,6691826522.10
260,"Yemen, Rep.",YEM,26910851361.76,25130274124.25,30906749533.22,32726417878.39
261,South Africa,ZAF,286769029768.39,295937388335.71,375348316917.75,416417032178.54
262,Zambia,ZMB,17910858637.90,15328342303.96,20265556273.58,23460098339.75


In [39]:
#Cleaning data to include only country, country code, and applicable years for Life Exp.
life_exp_df = life_expectancy.loc[:,["Country Name", "Country Code","2008","2009","2010","2011"]]
life_exp_df=life_exp_df.round(2)
life_exp_df=life_exp_df.dropna()
life_exp_df.head()

,Country Name,Country Code,2008,2009,2010,2011
0,Aruba,ABW,74.72,74.87,75.02,75.16
1,Afghanistan,AFG,59.93,60.48,61.03,61.55
2,Angola,AGO,53.24,54.31,55.35,56.33
3,Albania,ALB,75.91,76.22,76.56,76.91
5,Arab World,ARB,69.94,70.16,70.35,70.53


In [40]:
life_exp_df.dtypes

Country Name     object
Country Code     object
2008            float64
2009            float64
2010            float64
2011            float64
dtype: object

In [41]:
gdp_df.dtypes

Country Name     object
Country Code     object
2008            float64
2009            float64
2010            float64
2011            float64
dtype: object

In [43]:
#Aggregating/Merging GDP and Life Expectancy into one Data Frame
merged_df = pd.merge(gdp_df, life_exp_df, on = "Country Name")
merged_df

,Country Name,Country Code_x,2008_x,2009_x,2010_x,2011_x,Country Code_y,2008_y,2009_y,2010_y,2011_y
0,Aruba,ABW,2745251396.65,2498882681.56,2390502793.30,2549720670.39,ABW,74.72,74.87,75.02,75.16
1,Afghanistan,AFG,10109218067.79,12439087076.77,15856574731.44,17804280538.11,AFG,59.93,60.48,61.03,61.55
2,Angola,AGO,88538611205.14,70307163678.24,83799496611.20,111789686464.44,AGO,53.24,54.31,55.35,56.33
3,Albania,ALB,12881353507.85,12044208085.86,11926953213.70,12890866742.65,ALB,75.91,76.22,76.56,76.91
4,Arab World,ARB,2078823967037.02,1796358749934.16,2110062113014.68,2501833645178.18,ARB,69.94,70.16,70.35,70.53
...,...,...,...,...,...,...,...,...,...,...,...
233,Kosovo,XKX,5687417606.56,5653792720.20,5830463576.16,6691826522.10,XKX,69.40,69.65,69.90,70.15
234,"Yemen, Rep.",YEM,26910851361.76,25130274124.25,30906749533.22,32726417878.39,YEM,64.89,65.26,65.55,65.77
235,South Africa,ZAF,286769029768.39,295937388335.71,375348316917.75,416417032178.54,ZAF,55.36,56.46,57.67,58.90
236,Zambia,ZMB,17910858637.90,15328342303.96,20265556273.58,23460098339.75,ZMB,52.61,54.13,55.66,57.13


In [44]:
merged_df.columns

Index(['Country Name', 'Country Code_x', '2008_x', '2009_x', '2010_x',
       '2011_x', 'Country Code_y', '2008_y', '2009_y', '2010_y', '2011_y'],
      dtype='object')

In [53]:
#Cleaning up column names and removing extra country code column
gdp_life_df = merged_df.rename(columns={'Country Name': "Country Name",
                                        'Country Code_x': "Country Code", 
                                        '2008_x':"2008 GDP in Dollars", 
                                        '2009_x':"2009 GDP in Dollars",
                                        '2010_x':"2010 GDP in Dollars",
                                        '2011_x':"2011 GDP in Dollars", 
                                        'Country Code_y': "Country Code 2", 
                                        '2008_y': "2008 Life Expectancy in Years", 
                                        '2009_y': "2009 Life Expectancy in Years", 
                                        '2010_y': "2010 Life Expectancy in Years", 
                                        '2011_y': "2011 Life Expectancy in Years"
    
})
del gdp_life_df["Country Code 2"]
gdp_life_df.head()

,Country Name,Country Code,2008 GDP in Dollars,2009 GDP in Dollars,2010 GDP in Dollars,2011 GDP in Dollars,2008 Life Expectancy in Years,2009 Life Expectancy in Years,2010 Life Expectancy in Years,2011 Life Expectancy in Years
0,Aruba,ABW,2745251396.65,2498882681.56,2390502793.30,2549720670.39,74.72,74.87,75.02,75.16
1,Afghanistan,AFG,10109218067.79,12439087076.77,15856574731.44,17804280538.11,59.93,60.48,61.03,61.55
2,Angola,AGO,88538611205.14,70307163678.24,83799496611.20,111789686464.44,53.24,54.31,55.35,56.33
3,Albania,ALB,12881353507.85,12044208085.86,11926953213.70,12890866742.65,75.91,76.22,76.56,76.91
4,Arab World,ARB,2078823967037.02,1796358749934.16,2110062113014.68,2501833645178.18,69.94,70.16,70.35,70.53


In [54]:
#Creating new output files to be used within SQL
gdp_df.to_csv("Output/gdp_table.csv", index=False, header=True)
life_exp_df.to_csv("Output/life_exp_table.csv", index=False, header=True)
gdp_life_df.to_csv("Output/gdp_life_table.csv", index=False, header=True)

In [55]:
#Create Engine
rds_connection_string = "postgres:Edward1290!@localhost:5432/gdp_life_DB"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [56]:
engine.table_names()

['gdp', 'life_expectancy', 'gdp_life']

In [60]:
#Import to SQL - GDP
gdp_df.to_sql(name='gdp', con=engine, if_exists='append', index=False)

In [61]:
#Import to SQL - Life Expectancy
life_exp_df.to_sql(name='life_expectancy', con=engine, if_exists='append', index=False)

In [59]:
#Import to SQL - GDP AND Life Expectancy
gdp_life_df.to_sql(name='gdp_life', con=engine, if_exists='append', index=False)

In [62]:
#Select From check for GDP
pd.read_sql_query('select * from gdp', con=engine).head()

,id,Country Name,Country Code,2008,2009,2010,2011
0,1,Aruba,ABW,2745251396.64804,2498882681.56425,2390502793.2960896,2549720670.3910604
1,2,Afghanistan,AFG,10109218067.7904,12439087076.7667,15856574731.4411,17804280538.1102
2,3,Angola,AGO,88538611205.1433,70307163678.23799,83799496611.2004,111789686464.441
3,4,Albania,ALB,12881353507.8539,12044208085.863998,11926953213.6961,12890866742.6533
4,5,Andorra,AND,4007353156.58415,3660530702.97305,3355695364.23841,3442062830.13622


In [63]:
#Select From check for Life Expectancy
pd.read_sql_query('select * from life_expectancy', con=engine).head()

,id,Country Name,Country Code,2008,2009,2010,2011
0,1,Aruba,ABW,74.72,74.87,75.02,75.16
1,2,Afghanistan,AFG,59.93,60.48,61.03,61.55
2,3,Angola,AGO,53.24,54.31,55.35,56.33
3,4,Albania,ALB,75.91,76.22,76.56,76.91
4,5,Arab World,ARB,69.94,70.16,70.35,70.53


In [64]:
#Select From check for GDP and Life Expectancy
pd.read_sql_query('select * from gdp_life', con=engine).head()

,id,Country Name,Country Code,2008 GDP in Dollars,2009 GDP in Dollars,2010 GDP in Dollars,2011 GDP in Dollars,2008 Life Expectancy in Years,2009 Life Expectancy in Years,2010 Life Expectancy in Years,2011 Life Expectancy in Years
0,1,Aruba,ABW,2745251396.64804,2498882681.56425,2390502793.2960896,2549720670.3910604,74.72,74.87,75.02,75.16
1,2,Afghanistan,AFG,10109218067.7904,12439087076.7667,15856574731.4411,17804280538.1102,59.93,60.48,61.03,61.55
2,3,Angola,AGO,88538611205.1433,70307163678.23799,83799496611.2004,111789686464.441,53.24,54.31,55.35,56.33
3,4,Albania,ALB,12881353507.8539,12044208085.863998,11926953213.6961,12890866742.6533,75.91,76.22,76.56,76.91
4,5,Arab World,ARB,2078823967037.02,1796358749934.16,2110062113014.68,2501833645178.18,69.94,70.16,70.35,70.53
